In [2]:
import meld_graph
import meld_graph.models
import meld_graph.experiment
import meld_graph.dataset
import meld_graph.data_preprocessing
import meld_graph.evaluation


import importlib
importlib.reload(meld_graph)
importlib.reload(meld_graph.models)
importlib.reload(meld_graph.dataset)
importlib.reload(meld_graph.experiment)
importlib.reload(meld_graph.data_preprocessing)
importlib.reload(meld_graph.evaluation)

import logging
import os
import json

from meld_graph.dataset import GraphDataset, Oversampler
from meld_classifier.meld_cohort import MeldCohort, MeldSubject
from meld_graph.training import Metrics
import numpy as np
from meld_graph.paths import EXPERIMENT_PATH

from meld_graph.evaluation import Evaluator



### Generate dataset

In [6]:
#initialise dataset
cohort =  MeldCohort(hdf5_file_root='{site_code}_{group}_featurematrix_combat_6.hdf5')
subject_ids = cohort.get_subject_ids(group='all', dataset='MELD_dataset_V6_test.csv' )

#test on subject
site_codes=['H4']
cohort= MeldCohort(hdf5_file_root='{site_code}_{group}_featurematrix.hdf5')
subjects = ['MELD_H4_3T_FCD_0011']

features=  ['.on_lh.gm_FLAIR_0.25.mgh',
             '.on_lh.gm_FLAIR_0.5.mgh',
             '.on_lh.gm_FLAIR_0.75.mgh',
            '.on_lh.thickness.mgh']
            
# initiate params
params = {                
          "augment_data": {}, 
          "lesion_bias": 0,
          'synthetic_data': {
            'run_synthetic':False,
            'n_subs': 20,
            'use_controls':True,
            'bias': 1,
            'radius':0.5,
            'n_subtypes':1,
            'proportion_features_abnormal':0.9,
            'proportion_hemispheres_lesional':0.9,
            'jitter_factor':2, 
            },
          "preprocessing_parameters": {
              "scaling": None, 
              "zscore": True,
            },
          "combine_hemis": None,
          "lobes": False,
          "lesion_bias": 0,
          }

dataset = GraphDataset(subjects, cohort, params )


KeyError: 'features'

In [2]:
#load experiment already trained using checkpoint path

EXPERIMENT_PATH='/rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1'
checkpoint_path=os.path.join(EXPERIMENT_PATH, '22-08-22_synth_unet_gamma_2_synthetic_data__bias/synthetic_data__bias_1/fold_00')

exp = meld_graph.experiment.Experiment.from_folder(checkpoint_path)


Initialised Experiment 22-08-22_synth_unet_gamma_2_synthetic_data__bias/synthetic_data__bias_1


In [3]:
# update experiment with the test data parameters you want to run on
# here trying with synthetic data
exp.data_parameters['synthetic_data']['n_subs'] = 8

In [4]:
# Run the evaluation on the test data and save into directory provided
# Save images and evaluation metrics in path provided

save_dir=os.path.join(EXPERIMENT_PATH,'22-08-22_synth_eval_bis')

eva = Evaluator(experiment = exp,
                checkpoint_path = checkpoint_path,
                save_dir = save_dir ,
                make_images = True,
               )

eva.evaluate()


Using coord type exact
Loading model weights from checkpoint /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1/22-08-22_synth_unet_gamma_2_synthetic_data__bias/synthetic_data__bias_1/fold_00/best_model.pt


conv 33 16
conv 16 16
conv 16 16


### Evaluation on real data

In [2]:
#load experiment already trained using checkpoint path

EXPERIMENT_PATH='/rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1'
checkpoint_path=os.path.join(EXPERIMENT_PATH, '22-08-22_synth_unet_gamma_2_synthetic_data__bias/synthetic_data__bias_1/fold_00')

exp = meld_graph.experiment.Experiment.from_folder(checkpoint_path)

Initialised Experiment 22-08-22_synth_unet_gamma_2_synthetic_data__bias/synthetic_data__bias_1


In [6]:
# update experiment with the test data parameters you want to run on
# here trying with real data

exp.data_parameters['hdf5_file_root'] = '{site_code}_{group}_featurematrix_combat_6.hdf5'
exp.data_parameters['site_codes'] = ['H4']
exp.data_parameters['scanners'] = ['3T']
exp.data_parameters['dataset'] = 'MELD_dataset_V6_test.csv'
exp.data_parameters['synthetic_data'] = False

In [8]:
# Run the evaluation on the test data and save into directory provided
# Save images and evaluation metrics in path provided

save_dir=os.path.join(EXPERIMENT_PATH,'22-08-22_real_eval')

eva = Evaluator(experiment = exp,
                checkpoint_path = checkpoint_path,
                save_dir = save_dir ,
                make_images = True,
               )

eva.evaluate()


total number of subjects: 46
total number of subjects after restricting to subjects from MELD_dataset_V6.csv: 46
total number after filtering by scanner ['3T'], features, lesional_only True: 18
Using coord type exact
Loading model weights from checkpoint /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1/22-08-22_synth_unet_gamma_2_synthetic_data__bias/synthetic_data__bias_1/fold_00/best_model.pt
loading data and predicting model
Loading and preprocessing data
Combine hemis None


conv 33 16
conv 16 16
conv 16 16


TypeError: 'bool' object is not subscriptable